In [1]:
# STEP 1: Getting the pre-trained Model

# Install necessary packages
! pip install demucs
! pip install torchaudio
! pip install os
! pip install librosa
! pip install np
! pip install musdb
! pip install stempeg

# Need to install fmpeg outside the console. Use conda install -c conda-forge ffmpeg

from demucs.pretrained import get_model

# Load the pre-trained Demucs model
model_name = "htdemucs"  # Change to the specific model name you wish to use
separator = get_model(model_name)

# Verifying the model is loaded
print(separator)

ERROR: Could not find a version that satisfies the requirement os (from versions: none)
ERROR: No matching distribution found for os



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\mikea\miniconda3\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\Users\mikea\miniconda3\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "c:\Users\mikea\miniconda3\Lib\site-packages\ipykernel\kernelapp.py", line 739, in start
    self.io_loop.start()
  File "c:\Users\mikea\miniconda3\Lib\site-

BagOfModels(
  (models): ModuleList(
    (0): HTDemucs(
      (encoder): ModuleList(
        (0): HEncLayer(
          (conv): Conv2d(4, 48, kernel_size=(8, 1), stride=(4, 1), padding=(2, 0))
          (norm1): Identity()
          (rewrite): Conv2d(48, 96, kernel_size=(1, 1), stride=(1, 1))
          (norm2): Identity()
          (dconv): DConv(
            (layers): ModuleList(
              (0): Sequential(
                (0): Conv1d(48, 6, kernel_size=(3,), stride=(1,), padding=(1,))
                (1): GroupNorm(1, 6, eps=1e-05, affine=True)
                (2): GELU(approximate='none')
                (3): Conv1d(6, 96, kernel_size=(1,), stride=(1,))
                (4): GroupNorm(1, 96, eps=1e-05, affine=True)
                (5): GLU(dim=1)
                (6): LayerScale()
              )
              (1): Sequential(
                (0): Conv1d(48, 6, kernel_size=(3,), stride=(1,), padding=(2,), dilation=(2,))
                (1): GroupNorm(1, 6, eps=1e-05, affine=True)
  

In [2]:
# STEP 2: Normalize and Preprocess the Data

import os
import subprocess
import librosa
import numpy as np

# Function to extract audio stems from .mp4 file using ffmpeg
def extract_stems(mp4_path, extract_dir):
    stems = ['mixture', 'drums', 'bass', 'other', 'vocals']
    os.makedirs(extract_dir, exist_ok=True)
    for stem in stems:
        stem_path = os.path.join(extract_dir, f"{stem}.wav")
        command = ['ffmpeg', '-i', mp4_path, '-map', f'0:a:{stems.index(stem)}', stem_path]
        subprocess.run(command, check=True)
    print(f"Extracted stems from {mp4_path}")

# Normalize and convert audio to spectrogram functions
def normalize_audio(audio, orig_sr, target_sr=22050):
    audio_resampled = librosa.resample(audio, orig_sr=orig_sr, target_sr=target_sr)
    return audio_resampled

def audio_to_spectrogram(audio, sr=22050, n_fft=2048, hop_length=512):
    stft = librosa.stft(audio, n_fft=n_fft, hop_length=hop_length)
    spectrogram = librosa.amplitude_to_db(np.abs(stft))
    return spectrogram

# Helper function to process and save spectrogram
def process_and_save(audio_path, save_spect_dir, target_sr=22050):
    audio, _ = librosa.load(audio_path, sr=None)
    audio_norm = normalize_audio(audio, target_sr)
    spectrogram = audio_to_spectrogram(audio_norm, sr=target_sr)
    
    # Create the directory if it doesn't exist
    os.makedirs(save_spect_dir, exist_ok=True)
    
    # Save the spectrogram as a numpy file
    spectrogram_path = os.path.join(save_spect_dir, os.path.basename(audio_path).replace('.wav', '.npy'))
    np.save(spectrogram_path, spectrogram)
    print(f"Saved spectrogram: {spectrogram_path}")

# Path to MUSDB18
musdb18_path = '../musdb18'
save_spect_path = '../outputs/spectrograms_musdb18'

# Process each song in the dataset
for subset in ['train', 'test']:
    subset_dir = os.path.join(musdb18_path, subset)
    save_subset_dir = os.path.join(save_spect_path, subset)
    
    for file in os.listdir(subset_dir):
        if file.endswith('.stem.mp4'):
            mp4_path = os.path.join(subset_dir, file)
            track_name = os.path.splitext(file)[0]
            extract_dir = os.path.join(subset_dir, track_name)
            
            # Extract audio stems
            extract_stems(mp4_path, extract_dir)
            
            # Process and save spectrograms for each stem
            for stem in ['mixture', 'drums', 'bass', 'other', 'vocals']:
                stem_path = os.path.join(extract_dir, f"{stem}.wav")
                if os.path.exists(stem_path):
                    process_and_save(stem_path, save_subset_dir)

print("Preprocessing complete.")

Extracted stems from ../musdb18\train\A Classic Education - NightOwl.stem.mp4
Saved spectrogram: ../outputs/spectrograms_musdb18\train\mixture.npy
Saved spectrogram: ../outputs/spectrograms_musdb18\train\drums.npy
Saved spectrogram: ../outputs/spectrograms_musdb18\train\bass.npy
Saved spectrogram: ../outputs/spectrograms_musdb18\train\other.npy
Saved spectrogram: ../outputs/spectrograms_musdb18\train\vocals.npy
Extracted stems from ../musdb18\train\Actions - Devil's Words.stem.mp4
Saved spectrogram: ../outputs/spectrograms_musdb18\train\mixture.npy
Saved spectrogram: ../outputs/spectrograms_musdb18\train\drums.npy
Saved spectrogram: ../outputs/spectrograms_musdb18\train\bass.npy
Saved spectrogram: ../outputs/spectrograms_musdb18\train\other.npy
Saved spectrogram: ../outputs/spectrograms_musdb18\train\vocals.npy
Extracted stems from ../musdb18\train\Actions - One Minute Smile.stem.mp4
Saved spectrogram: ../outputs/spectrograms_musdb18\train\mixture.npy
Saved spectrogram: ../outputs/spec